In [1]:
import os
data_path = os.environ["YAHOO_DATA"]
print data_path
rawEventsRdd = sc.textFile(data_path + "events.txt")

/home/mertergun/yahoo_data/


In [3]:
sc.parallelize(rawEventsRdd.take(1000)).cache().saveAsTextFile(data_path + "filteredEvents.txt")

In [2]:
rawEventsRdd = sc.textFile(data_path + "filteredEvents.txt")
execfile("../script/context.py")
context

[<function __main__.context_timestamp>,
 <function __main__.context_latitude>,
 <function __main__.context_longitude>,
 <function __main__.context_city>,
 <function __main__.context_dayOfWeek>,
 <function __main__.context_time_of_day>,
 <function __main__.context_event_type>]

In [3]:
userIdConversionDictionary = rawEventsRdd.map(get_user_id).distinct().zipWithIndex().collectAsMap()
userIdConversionDictionaryBroadcast = sc.broadcast(userIdConversionDictionary)
itemIdConversionDictionary = rawEventsRdd.map(get_item_id).distinct().zipWithIndex().collectAsMap()
itemIdConversionDictionaryBroadcast = sc.broadcast(itemIdConversionDictionary)
cityConversionDictionary = rawEventsRdd.map(context_city).distinct().zipWithIndex().collectAsMap()
cityConversionDictionaryBroadcast = sc.broadcast(cityConversionDictionary)

In [9]:
userIdConversionDictionary

{u'001e6d8e-cbe7-4374-8c38-f37962a457e9': 1,
 u'005ca0a2-6e94-4709-adc6-f35cdd50bf8f': 0}

In [5]:
def set_context(line):
    key1 = (userIdConversionDictionary[get_user_id(line)], 
            itemIdConversionDictionary[get_item_id(line)])
    list = []
    for f in context:
        list = list + [(context.index(f), f(line))]
    #convert city to id
    list[3] = list[3][0], cityConversionDictionary[list[3][1]]
    return [key1,list]
   
eventRDDConverted = rawEventsRdd.map(set_context).cache()
eventRDDConverted.take(3)

[[(1, 41),
  [(0, u'1421521691'),
   (1, u'47.235050'),
   (2, u'-122.534698'),
   (3, 9),
   (4, u'5'),
   (5, u'11'),
   (6, u'App_Opened')]],
 [(1, 46),
  [(0, u'1421558502'),
   (1, u'47.235050'),
   (2, u'-122.534698'),
   (3, 9),
   (4, u'5'),
   (5, u'21'),
   (6, u'App_Opened')]],
 [(1, 44),
  [(0, u'1421017853'),
   (1, u'47.237461'),
   (2, u'-122.530899'),
   (3, 9),
   (4, u'6'),
   (5, u'15'),
   (6, u'App_Opened')]]]

In [6]:
def flat_context(lst):
    l = []
    for x in lst[1]:
        l = l + [[(lst[0][0], lst[0][1], x[0]), x[1]]]
    return l
eventRDD = eventRDDConverted.flatMap(flat_context).cache()
eventRDD.take(3)

[[(1, 41, 0), u'1421521691'],
 [(1, 41, 1), u'47.235050'],
 [(1, 41, 2), u'-122.534698']]